# Convert results from PCC-O3D-ML to LAS format
This notebook allows users to take the binary result labels and create a folder with the results and ground truth saved as LAS files. Open3D-ML also features vizualization scripts which are configured with PCC_SKITTI in the binary format, but since the original Point Cloud Dataset was originally in LAS files this script provides seamless input to output.

The input format structure expected should match that of the Semantic KITTI dataset.

In [2]:
import numpy as np
import os
import laspy

In [6]:
# Add paths
pcc_skitti = 'add path to PCC_SKITTI' #(find instructions in /PointCloudCity-Open3D-ML/pscr_point_cloud_city)
o3dml_results = 'add path to test results' #(found in /PointCloudCity-Open3D-ML/test/)
output_dir = # path to output folder

filenames = os.listdir(o3dml_results)

In [5]:
for seq in filenames:

    # PATHS
    seq_p = os.path.join(pcc_skitti, seq)
    seq_l = os.path.join(o3dml_results, seq)

    # PCC Points
    path_vel = 'velodyne'
    path_points_1 = os.path.join(seq_p, path_vel)
    path_bin = '000000.bin'
    path_points = os.path.join(path_points_1, path_bin)
    scan = np.fromfile(path_points, dtype='float32')

    # Ground Truth Labels
    path_ll = 'labels'
    path_l = os.path.join(path_ll, '000000.label')
    path_labels = os.path.join(seq_p, path_l)

    # Results Labels
    path_lr = 'predictions'
    path_lrl = os.path.join(path_lr, '000000.label')
    path_results = os.path.join(seq_l, path_lrl)

    # Transform points
    pntz = scan.reshape((-1, 4))
    points = np.delete(pntz, [3], axis=1)

    # Load ground truth and results labels
    ground_labels_un = np.fromfile(path_labels, dtype='uint32')
    results_labels_un = np.fromfile(path_results, dtype='uint32')

    gt_saver = ground_labels_un
    res_saver = results_labels_un
    pnt_saver = points

    points = pnt_saver
    ground_labels = gt_saver
    results_labels = res_saver

    #SAVE PATHS
    save_gt = seq+'_gt.las'
    save_results = seq+'_results.las'
    save_gt_path = os.path.join(output_dir, save_gt)
    save_results_path = os.path.join(output_dir, save_results)

    # ### SAVE GT
    header = laspy.LasHeader(point_format=3, version="1.4")
    header.offsets = np.min(points, axis=0)
    header.scales = np.array([0.1, 0.1, 0.1])
    with laspy.open(save_gt_path, mode="w", header=header) as writer:
        point_record = laspy.ScaleAwarePointRecord.zeros(points.shape[0], header=header)
        point_record.x = points[:, 0]
        point_record.y = points[:, 1]
        point_record.z = points[:, 2]
        point_record.classification = ground_labels[:]

        writer.write_points(point_record)

    ### SAVE RESULTS
    with laspy.open(save_results_path, mode="w", header=header) as writer:
        point_record = laspy.ScaleAwarePointRecord.zeros(points.shape[0], header=header)
        point_record.x = points[:, 0]
        point_record.y = points[:, 1]
        point_record.z = points[:, 2]
        point_record.classification = results_labels[:]

        writer.write_points(point_record)

    # CLEAN
    del pntz, scan, points, ground_labels_un, results_labels_un, ground_labels, results_labels 